# 基本数据文件处理，（age_train,age_test,user_basic_info,user_habavior_info）

## 获得752维度的训练集(（trainData752.npz）)和测试集（testData752.npz）

In [1]:
import numpy as np
import pandas as pd
import gc
import re
from time import strftime, localtime
from scipy import sparse
from keras.utils import np_utils
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from scipy.sparse import csr_matrix, hstack
from tqdm import tqdm
import printTime as pt
# pd.set_option('display.max_columns',20)

Using TensorFlow backend.


In [2]:
# useRowNum = 1000000

In [3]:
def do_partOne():
    age_train_data = pd.read_csv( r'../data/age_train.csv',header=None,names=['uId','age_group'])
    # print(age_train_data.shape)
    # print(age_train_data.head())
    
    # print('age_group缺失值数量为：',age_train_data['age_group'].isnull().sum())
    # print(age_train_data['age_group'].value_counts())
    age = age_train_data['age_group']
    age_train_data.drop(labels='age_group',inplace=True,axis=1)
    
    user_basic_info_data = pd.read_csv( r'../data/user_basic_info.csv',header=None,names=['uId','gender','city','prodName','ramCapacity','ramLeftRation',
                                                                                        'romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
    # print(user_basic_info_data.head())
    
    one_hot_feature = pd.DataFrame(user_basic_info_data.values[:,[0,2,3,8,10,11]])
    user_basic_info_data.drop(labels=['city','prodName','color','ct','carrier'],axis=1,inplace=True)
    
    # 缺失值填充
    imp = SimpleImputer(missing_values=np.nan,strategy='mean')
    user_basic_info_data = pd.DataFrame(imp.fit_transform(user_basic_info_data.values))
    # print(user_basic_info_data.head())
    # print(user_basic_info_data.shape)
    
    user_basic_info_data = user_basic_info_data.values
    
    scale = StandardScaler()
    user_basic_info_data[:,1:8] = scale.fit_transform(user_basic_info_data[:,1:8])
    user_basic_info_data = pd.DataFrame(user_basic_info_data)
    # user_basic_info_data.head()
    
    # 标签编码
    city = pd.factorize(values=one_hot_feature[1]); one_hot_feature[1] = city[0]
    prodName = pd.factorize(values=one_hot_feature[2]); one_hot_feature[2] = prodName[0]
    color = pd.factorize(values=one_hot_feature[3]); one_hot_feature[3] = color[0]
    ct = pd.factorize(values=one_hot_feature[4]); one_hot_feature[4] = ct[0]
    carrier = pd.factorize(values=one_hot_feature[5]); one_hot_feature[5] = carrier[0]
    
    imp = SimpleImputer(missing_values=-1,strategy='mean')
    one_hot_feature = pd.DataFrame(imp.fit_transform(one_hot_feature.values)).astype(np.int)
    # print(one_hot_feature.shape)
    # print(one_hot_feature.head())
    pt.printTime()
    one_hot_city = np_utils.to_categorical(one_hot_feature[1],num_classes=len(one_hot_feature[1].value_counts()))
    one_hot_prodName = np_utils.to_categorical(one_hot_feature[2],num_classes=len(one_hot_feature[2].value_counts()))
    one_hot_color = np_utils.to_categorical(one_hot_feature[3],num_classes=len(one_hot_feature[3].value_counts()))
    one_hot_ct = np_utils.to_categorical(one_hot_feature[4],num_classes=len(one_hot_feature[4].value_counts()))
    one_hot_carrier = np_utils.to_categorical(one_hot_feature[5],num_classes=len(one_hot_feature[5].value_counts()))
    pt.printTime()
    
    one_hot_feature = pd.DataFrame(np.column_stack((user_basic_info_data[0].values,one_hot_city,one_hot_prodName,one_hot_color,one_hot_ct,one_hot_carrier)))
    
    user_behavior_info_data = pd.read_csv(r'../data/user_behavior_info.csv',header=None)
    # print(user_behavior_info_data.head());print(user_behavior_info_data.shape)
    # user_behavior_info_data.drop(columns=['alarmClock','frontCameraTimes','backCameraTimes','callOutTimes','callInTimes','callVoiceTimes'],inplace=True)
    # imp = SimpleImputer(missing_values=0,strategy='mean')
    # tmp = imp.fit_transform(user_behavior_info_data.values)
    # user_behavior_info_data = pd.DataFrame(tmp,columns=['uId','bootTimes','alarmClock','frontCameraTimes','backCameraTimes','callOutTimes','callInTimes','callVoiceTimes','totalTraffic'])
    # user_behavior_info_data.dropna(axis=0,how='any',inplace=True)
    # print(user_behavior_info_data.head())
    # print(user_behavior_info_data.shape)
    # print(user_behavior_info_data.describe())
    
    user_behavior_info_data = user_behavior_info_data.values
    scale = StandardScaler()
    user_behavior_info_data[:,1:9] = scale.fit_transform(user_behavior_info_data[:,1:9])
    user_behavior_info_data = pd.DataFrame(user_behavior_info_data)
    # user_behavior_info_data.head()
    
    age_train_data = pd.merge(age_train_data,user_basic_info_data,how='inner',left_on='uId',right_on=0)
    age_train_data = pd.merge(age_train_data,user_behavior_info_data,how='inner',left_on='uId',right_on=0)
    age_train_data = pd.merge(age_train_data,one_hot_feature,how='inner',left_on='uId',right_on=0)
    pt.printTime()
    # print(age_train_data.shape)
    # print(age_train_data.head())
    
    age_train_data.drop(labels=['uId','0_x','0_y',0],axis=1,inplace=True)
    age_train_data = sparse.csr_matrix(age_train_data)
    age_train_data.shape
    sparse.save_npz(r'../trainTestData/trainData752.npz',age_train_data)
    
    age_test_data = pd.read_csv(r'../data/age_test.csv',header=None,names=['uId'])
    age_test_data = pd.merge(age_test_data,user_basic_info_data,how='inner',left_on='uId',right_on=0)
    age_test_data = pd.merge(age_test_data,user_behavior_info_data,how='inner',left_on='uId',right_on=0)
    age_test_data = pd.merge(age_test_data,one_hot_feature,how='inner',left_on='uId',right_on=0)
    # print(age_test_data.shape)
    # print(age_test_data.head())
    
    age_test_data.drop(labels=['uId','0_x','0_y',0],axis=1,inplace=True)
    age_test_data.shape
    age_test_data = sparse.csr_matrix(age_test_data)
    age_test_data.shape
    sparse.save_npz(r'../trainTestData/testData752.npz',age_test_data)

In [4]:
pt.printTime()
do_partOne()
pt.printTime()

2019-08-20 20:55:38
2019-08-20 20:55:48
2019-08-20 20:55:51
2019-08-20 20:56:58
2019-08-20 20:58:08


# 提取actived中重要的app

In [5]:
def get_importAppActivate():
    age_train = pd.read_csv('../data/age_train.csv',header=None)
    user_app_actived = pd.read_csv('../data/user_app_actived.csv',header=None)
    
    appNum,i = {}, 0 
    for user_info in tqdm(user_app_actived.values):
        appIdList = user_info[1].split('#')
        if appIdList[0]!='\\N':
            for app in appIdList:
                if app not in appNum.keys():
                    appNum[app] = 0
                else:
                    appNum[app] += 1
                    
    appNumList = np.array(list(appNum.values()))
    boolNumList = appNumList>500
    appNumMore500 = np.array(list(appNum.keys()))[boolNumList]
    
    user_app_actived = pd.merge(age_train,user_app_actived,how='inner',on=0)
    
    col_index = {} #{appId:column}
    i=0
    for user_info in tqdm(user_app_actived.values):
        appIdList = re.split('#',user_info[2])
        if appIdList[0]!='\\N':
            for app in appIdList:
                if app not in col_index.keys():
                    col_index[app]=i
                    i+=1
                    
    age_appId_number_matrix = np.zeros((6,len(col_index.keys())))
    
    for user_info in tqdm(user_app_actived.values):
        appIdList = re.split('#',user_info[2])
        if appIdList[0]!='\\N':
            for app in appIdList:
                col = col_index[app]
                row = user_info[1]-1
                age_appId_number_matrix[row][col] += 1
                
    ageNumber = age_train[1].value_counts()
    
    age_appId_number_matrix[0] = age_appId_number_matrix[0]/ageNumber[1]
    age_appId_number_matrix[1] = age_appId_number_matrix[1]/ageNumber[2]
    age_appId_number_matrix[2] = age_appId_number_matrix[2]/ageNumber[3]
    age_appId_number_matrix[3] = age_appId_number_matrix[3]/ageNumber[4]
    age_appId_number_matrix[4] = age_appId_number_matrix[4]/ageNumber[5]
    age_appId_number_matrix[5] = age_appId_number_matrix[5]/ageNumber[6]
    
    age_appId_number_matrix = pd.DataFrame(age_appId_number_matrix,columns=col_index.keys())
    
    app_stds = age_appId_number_matrix.std()
    app_stds.sort_values(ascending=False,inplace=True)
    
    age_appId_number_matrix = age_appId_number_matrix.reindex(columns=app_stds.keys())
    
    app_keys = age_appId_number_matrix.keys()
    
    appMore500AndStd = []
    for app in app_keys:
        if app in appNumMore500:
            appMore500AndStd.append(app)
            
    importance_app = appMore500AndStd[:4000]
    
    importance_app = pd.DataFrame(importance_app)
    importance_app.to_csv('../processed/importance_app_inActivate.csv',header=None,index=False)
    importance_app.shape

In [6]:
pt.printTime()
get_importAppActivate()
pt.printTime()

2019-08-20 20:58:08


100%|██████████| 2010000/2010000 [01:21<00:00, 24574.26it/s]


2019-08-20 21:00:37


In [7]:
def do_dealActived():
    user_actived = pd.read_csv(r'../data/user_app_actived.csv',header=None)
    
    appList = {}
    for user in tqdm(user_actived.values):
        for app in user[1].split('#'):
            if app!='\\N':
                if app not in appList.keys():
                    appList[app] = 0
    
    age_train = pd.read_csv(r'../data/age_train.csv',header=None)
    age = age_train[1]
    
    user_row = {}
    i=0
    for user in age_train.values:
        user_row[user[0]] = i
        i += 1
    
    trainData = pd.merge(age_train,user_actived,how='inner',on=0)
    trainData.drop(columns=['1_x'],inplace=True,axis=1)
    
    all_user_app, user_app_train = [], []
    user_app = []
    batch_size, i=1000,0
    for user in tqdm(trainData.values):
    #     print(i)
        i += 1
        for app in user[1].split('#'):
            if app!='\\N':
                # [uId,appId,trainRow]
                user_app.append(user[0])
                user_app.append(app)
                user_app.append(user_row[user[0]])
                all_user_app.append(user_app)
                user_app = []
        if i == batch_size:
            all_user_app = pd.DataFrame(all_user_app)
            all_user_app.to_csv(r'../processed/user_app_train.csv',header=None,index=False,mode='a')
            user_app_train.append(all_user_app)
            all_user_app = []
            i= 0     
    all_user_app = pd.DataFrame(all_user_app)
    all_user_app.to_csv(r'../processed/user_app_train.csv',header=None,index=False,mode='a')
#     user_app_train.append(all_user_app)
    
    user_oneapp = pd.read_csv(r'../processed/user_app_train.csv',header=None)
    
    appencoder = LabelEncoder().fit(list(appList.keys()))
    user_oneapp[1] = appencoder.transform(user_oneapp[1])
    
    important_app = pd.read_csv(r'../processed/importance_app_inActivate.csv',header=None)
    important_app_rowIndex = appencoder.transform(important_app[0].values)
    important_app_rowIndex = pd.DataFrame(important_app_rowIndex)
    important_app_rowIndex.to_csv(r'../processed/important_app_rowIndex.csv',header=None,index=False)
    
    csr_trainData = csr_matrix((np.ones(user_oneapp.shape[0]),(user_oneapp[2],user_oneapp[1])),shape=(2010000,9400))
    sparse.save_npz(r'../trainTestData/trainData9400.npz',csr_trainData)
    csr_trainData = sparse.csr_matrix(csr_trainData[:,important_app_rowIndex[0].values])
    sparse.save_npz(r'../trainTestData/trainData4000.npz',csr_trainData)
    
    user_actived = pd.read_csv(r'../data/user_app_actived.csv',header=None)
    
    appList = {}
    for user in tqdm(user_actived.values):
        for app in user[1].split('#'):
            if app!='\\N':
                if app not in appList.keys():
                    appList[app] = 0
                    
    age_test = pd.read_csv(r'../data/age_test.csv',header=None)
    
    user_row = {}
    i=0
    for user in age_test.values:
        user_row[user[0]] = i
        i += 1
        
    testData = pd.merge(age_test,user_actived,how='inner',on=0)
    
    all_user_app,user_app_test = [],[]
    user_app = []
    batch_size, i=1000,0
    for user in tqdm(testData.values):
    #     print(i)
        i += 1
        for app in user[1].split('#'):
            if app!='\\N':
                # [uId,appId,trainRow]
                user_app.append(user[0])
                user_app.append(app)
                user_app.append(user_row[user[0]])
                all_user_app.append(user_app)
                user_app = []
        if i == batch_size:
            all_user_app = pd.DataFrame(all_user_app)
            all_user_app.to_csv(r'../processed/user_app_test.csv',header=None,index=False,mode='a')
#             user_app_test.append(all_user_app)
            all_user_app = []
            i = 0     
    all_user_app = pd.DataFrame(all_user_app)
    all_user_app.to_csv(r'../processed/user_app_test.csv',header=None,index=False,mode='a')
    user_app_test.append(all_user_app)
    
    user_oneapp = pd.read_csv(r'../processed/user_app_test.csv',header=None)
    appencoder = LabelEncoder().fit(list(appList.keys()))
    user_oneapp[1] = appencoder.transform(user_oneapp[1])
    
    csr_testData = csr_matrix((np.ones(user_oneapp.shape[0]),(user_oneapp[2],user_oneapp[1])),shape=(502500,9400))
    sparse.save_npz(r'../trainTestData/testData9400.npz',csr_testData)
    csr_testData = sparse.csr_matrix(csr_testData[:,important_app_rowIndex[0].values])
    sparse.save_npz(r'../trainTestData/testData4000.npz',csr_testData)

In [8]:
pt.printTime()
do_dealActived()
pt.printTime()

2019-08-20 21:00:37


100%|██████████| 502500/502500 [01:37<00:00, 5148.72it/s]


2019-08-20 21:12:59


In [9]:
# def do_appToCategory():
#     age_test_data = pd.read_csv(r'../data/age_test.csv',header=None,names=['uId'])
#     age_train_data = pd.read_csv( r'../data/age_train.csv',header=None,names=['uId','age_group'])
#     age = age_train_data['age_group']
#     age_train_data.drop(columns=['age_group'],inplace=True)
    
#     user_app_actived_data = pd.read_csv(r'../data/user_app_actived.csv',header=None,names=['uId','appId'])
    
#     user_numAppList = {} # {uId:[num,appList]}
#     for user in tqdm(user_app_actived_data.values):
#         appList = re.split(r'\#',user[1])
#         if appList[0] == '\\N':
#             user_numAppList[user[0]] = []
#             l = 0
#         else:
#             user_numAppList[user[0]] = appList
#             l = len(appList)
#         user_numAppList[user[0]].insert(0,l)
        
#     app_info_data = pd.read_csv(r'../data/app_info.csv',header=None,names=['appId','category'])
#     data = app_info_data['category'].value_counts(dropna=False)
#     app_category = {} #{appId:[category]}
#     for aid in app_info_data.values:
#         app_category.setdefault(aid[0],[]).append(aid[1])
        
#     category_num = {}
#     def initDict(categorylist):
#         for cate in categorylist:
#             category_num[cate] = 0
#         return category_num
#     categorylist = data.index
#     category_num = initDict(categorylist)
    
#     cate = list(np.zeros(40))
#     i = 1
#     batch_size = 10000 #每10000次存储一次数据
#     categorylist = data.index
#     print(len(categorylist),categorylist)
#     for numAppList in tqdm(user_numAppList.values()):
#         i += 1
#         category_num = initDict(categorylist)
#         if len(numAppList)==1:
#             cate = np.vstack((cate,list(category_num.values())))
#         else:
#             appList = numAppList[1:-1]
#             for appId in appList:
#                 if appId in app_category:
#                     cateList = app_category[appId]
#                     for category in cateList:
#                         category_num[category] += 1
#             cate = np.vstack((cate,list(category_num.values())))
#         if i%batch_size==0:
#             if i==batch_size:
#                 cate = pd.DataFrame(cate, columns=['实用工具', '便捷生活', '教育', '金融理财', '购物比价', '社交通讯', '影音娱乐', '新闻阅读', '休闲益智',
#                                                        '商务', '运动健康', '出行导航', '经营策略', '动作射击', '儿童', '角色扮演', '拍摄美化', '棋牌桌游',
#                                                        '旅游住宿', '汽车', '主题个性', '美食', '体育竞速', '网络游戏', '休闲游戏', '休闲娱乐', '动作冒险',
#                                                        '学习办公', '益智棋牌', '表盘个性', '电子书籍', '模拟游戏', '策略游戏', '棋牌天地', '体育射击',
#                                                        '图书阅读',
#                                                        '主题铃声', '角色游戏', '合作壁纸*', '医疗健康'])
#             else:
#                 cate = pd.DataFrame(cate)
#             cate.drop(0,,inplace=True)
#             if i==batch_size:
#                 cate.to_csv(r'../processed/category_num.csv', index=False, header=True, encoding='utf_8_sig',mode='a')
#             else:
#                 cate.to_csv(r'../processed/category_num.csv', index=False, header=False, encoding='utf_8_sig', mode='a')
#             cate = list(np.zeros(40))
#     cate = pd.DataFrame(cate)
#     cate.drop(0,inplace=True)
#     cate.to_csv(r'../processed/category_num.csv', index=False, header=False, encoding='utf_8_sig',mode='a')
    
#     category_num = pd.read_csv(r'../processed/category_num.csv')
    
#     col = pd.DataFrame(category_num.sum())
    
#     category_num = pd.DataFrame(category_num.values[:,col[0]>20])
    
#     user_app_actived = pd.read_csv(r'../data/user_app_actived.csv',header=None)
    
#     category_num = preprocessing.StandardScaler().fit_transform(category_num.values)
#     uId = user_app_actived[0].values
    
#     uId = uId.reshape(len(uId),1)
    
#     category_num = np.column_stack((uId,category_num))
    
#     category_num = pd.DataFrame(category_num)
    
#     age_train = pd.read_csv(r'../data/age_train.csv',header=None,usecols=[0])
#     age_test = pd.read_csv(r'../data/age_test.csv',header=None)

#     age_train = pd.merge(age_train,category_num,how='inner',on=0)

#     age_test = pd.merge(age_test,category_num,how='inner',on=0)

#     age_train.drop(labels=[0],axis=1,inplace=True)
#     age_test.drop(labels=[0],axis=1,inplace=True)

#     age_train = sparse.csr_matrix(age_train,dtype=np.float32)
#     age_test = sparse.csr_matrix(age_test,dtype=np.float32)

#     sparse.save_npz(r'../trainTestData/trainData30.npz',age_train)
#     sparse.save_npz(r'../trainTestData/testData30.npz',age_test)

# #     category_num.to_csv(r'../processed/category_num.csv',header=None,index=False)

In [10]:
# pt.printTime()
# do_appToCategory()
# pt.printTime()

2019-08-20 21:12:59


  0%|          | 1496/2512500 [00:00<02:47, 14948.50it/s]

40 Index(['实用工具', '便捷生活', '教育', '金融理财', '购物比价', '社交通讯', '影音娱乐', '新闻阅读', '休闲益智',
       '商务', '运动健康', '出行导航', '经营策略', '动作射击', '儿童', '角色扮演', '拍摄美化', '棋牌桌游',
       '旅游住宿', '汽车', '主题个性', '美食', '体育竞速', '网络游戏', '休闲游戏', '休闲娱乐', '动作冒险',
       '学习办公', '益智棋牌', '表盘个性', '模拟游戏', '策略游戏', '电子书籍', '棋牌天地', '体育射击', '图书阅读',
       '角色游戏', '主题铃声', '合作壁纸*', '医疗健康'],
      dtype='object')


100%|██████████| 2512500/2512500 [06:22<00:00, 6576.69it/s]
/home/admin1/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3183: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


ValueError: Found array with 0 feature(s) (shape=(5025001, 0)) while a minimum of 1 is required by StandardScaler.